In [118]:
import pandas as pd
from google.colab import drive

from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import tree
import numpy as np

!pip install --upgrade category_encoders
import category_encoders  as ce
from matplotlib import pyplot as plt

drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
data_frame = pd.read_csv("/content/drive/MyDrive/Organizacion De Datos/trabajo_practico_2/secondary_data.csv",sep=";")

Para aquellas filas con mucho porcentaje de nans ***(a mi criterio, las que tienen+60%)*** , voy a fillearlas con 'unknown', mientras que aquellas que tengan porcentaje bajo, voy a usar el **'most-frequent strategy'** de simpleImputer

In [120]:
(( len(data_frame) - data_frame.count() )*100 / len(data_frame)).nlargest(len(data_frame.columns))

veil-type               94.797688
spore-print-color       89.595376
veil-color              87.861272
stem-root               84.393064
stem-surface            62.427746
gill-spacing            41.040462
cap-surface             23.121387
gill-attachment         16.184971
ring-type                4.046243
class                    0.000000
stem-color               0.000000
habitat                  0.000000
has-ring                 0.000000
stem-width               0.000000
cap-diameter             0.000000
stem-height              0.000000
gill-color               0.000000
does-bruise-or-bleed     0.000000
cap-color                0.000000
cap-shape                0.000000
season                   0.000000
dtype: float64

# Separando la variable a predecir

In [121]:
X = data_frame.drop('class', axis=1)
y = data_frame['class']

In [122]:
y = y.map({"e":1,"p":0})

# Division de sets

In [123]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
X_validation, X_test, y_validation, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=13)

guardo el indice ya que despues de usar el SimpleImputer se me pierde ( lo necesito para mean encoding de category_encoders ) 

In [124]:
X_train_saved_index =  X_train.index
X_validation_saved_index = X_validation.index
X_test_saved_index = X_test.index

# Imputacion 

`Imputacion 1) Reemplazar por una constante`

In [125]:
X_train[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ] = X_train[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ].fillna('unknown')
X_validation[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ] = X_validation[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ].fillna('unknown')
X_test[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ] = X_test[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ].fillna('unknown')

`Imputacion 2) Reemplazar por el mas frecuente`

In [126]:
from sklearn.impute import SimpleImputer
simple = SimpleImputer(strategy='most_frequent')

In [127]:
X_train = pd.DataFrame(simple.fit_transform(X_train), columns = X_train.columns)
X_validation = pd.DataFrame(simple.transform(X_validation), columns = X_validation.columns)
X_test = pd.DataFrame(simple.transform(X_test), columns = X_test.columns)

acá vuelvo a pasar las medidas a float porque se me pasaron a objeto

In [128]:
X_train[ ['cap-diameter', 'stem-height', 'stem-width'] ] = X_train[ ['cap-diameter', 'stem-height', 'stem-width'] ].astype(float)
X_validation[ ['cap-diameter', 'stem-height', 'stem-width'] ] = X_validation[ ['cap-diameter', 'stem-height', 'stem-width'] ].astype(float)
X_test[ ['cap-diameter', 'stem-height', 'stem-width'] ] = X_test[ ['cap-diameter', 'stem-height', 'stem-width'] ].astype(float)

reemplazo el indice por el original

In [129]:
X_train.index = X_train_saved_index
X_validation.index = X_validation_saved_index 
X_test.index = X_test_saved_index

# Encodings 

Mean Encoding

In [130]:
meanEncod = ce.TargetEncoder(cols = ['cap-surface'	, 'gill-attachment'	])

In [131]:
X_train = meanEncod.fit_transform(X_train, y_train)
X_validation = meanEncod.transform(X_validation)
X_test =  meanEncod.transform(X_test)

OHE Encoding

In [132]:
ohe_ring = OneHotEncoder()

encoded_ring_train = ohe_ring.fit_transform(X_train[['has-ring']]).todense().astype(int)
encoded_ring_validation = ohe_ring.transform(X_validation[['has-ring']]).todense().astype(int)
encoded_ring_test = ohe_ring.transform(X_test[['has-ring']]).todense().astype(int)

encoded_bruise_train = ohe_ring.fit_transform(X_train[['does-bruise-or-bleed']]).todense().astype(int)
encoded_bruise_validation = ohe_ring.transform(X_validation[['does-bruise-or-bleed']]).todense().astype(int)
encoded_bruise_test = ohe_ring.transform(X_test[['does-bruise-or-bleed']]).todense().astype(int)

In [133]:
X_train['has-ring'] = encoded_ring_train
X_validation['has-ring'] = encoded_ring_validation
X_test['has-ring'] = encoded_ring_test

X_train['does-bruise-or-bleed'] = encoded_bruise_train
X_validation['does-bruise-or-bleed'] = encoded_bruise_validation
X_test['does-bruise-or-bleed'] = encoded_bruise_test

Binary Encoding

In [134]:
bin_encoded = ce.BinaryEncoder(cols=["cap-shape","cap-color","gill-spacing","gill-color","stem-root","stem-surface",
                                     "stem-color","veil-type","veil-color","ring-type","spore-print-color","habitat" ,"season"], return_df = True,)

X_train = bin_encoded.fit_transform(X_train)
X_validation = bin_encoded.transform(X_validation)
X_test = bin_encoded.transform(X_test)

# Keras 

In [135]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [136]:
X_train.shape

(48855, 51)

In [137]:
y_train.shape

(48855,)

Uso 'Sigmoid' como funcion de activacion ya que da la proba entre 0 y 1

In [138]:
tf.keras.utils.set_random_seed(13)
model = Sequential([
    Dense(10, activation="sigmoid", input_shape=(51,)),
    Dense(5, activation="sigmoid",input_shape=(51,)),
    Dense(1, activation="sigmoid",input_shape=(51,))
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 10)                520       
                                                                 
 dense_13 (Dense)            (None, 5)                 55        
                                                                 
 dense_14 (Dense)            (None, 1)                 6         
                                                                 
Total params: 581
Trainable params: 581
Non-trainable params: 0
_________________________________________________________________


y para la loss uso 'binary_crossentropy' ya que el problema es de clasificacion binaria

In [139]:
model.compile(loss = 'binary_crossentropy',metrics=["AUC"], optimizer = Adam(0.001))

In [140]:
model.fit(X_train, y_train, epochs = 10, batch_size = 32)

Epoch 1/10
1527/1527 [==============================] - 2s 1ms/step - loss: 0.6082 - auc: 0.7469
Epoch 2/10
1527/1527 [==============================] - 2s 1ms/step - loss: 0.4485 - auc: 0.8784
Epoch 3/10
1527/1527 [==============================] - 3s 2ms/step - loss: 0.3635 - auc: 0.9223
Epoch 4/10
1527/1527 [==============================] - 4s 2ms/step - loss: 0.2929 - auc: 0.9511
Epoch 5/10
1527/1527 [==============================] - 3s 2ms/step - loss: 0.2283 - auc: 0.9718
Epoch 6/10
1527/1527 [==============================] - 3s 2ms/step - loss: 0.1731 - auc: 0.9855
Epoch 7/10
1527/1527 [==============================] - 4s 2ms/step - loss: 0.1344 - auc: 0.9917
Epoch 8/10
1527/1527 [==============================] - 3s 2ms/step - loss: 0.1075 - auc: 0.9947
Epoch 9/10
1527/1527 [==============================] - 2s 1ms/step - loss: 0.0881 - auc: 0.9965
Epoch 10/10
1527/1527 [==============================] - 2s 1ms/step - loss: 0.0739 - auc: 0.9975


In [141]:
predicts = model.predict(X_validation)

191/191 [==============================] - 0s 800us/step


## Score para validacion

In [142]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_validation, predicts)

0.998149903775476

In [143]:
# 0.998149903775476

## Score para test

In [144]:
predicts_for_test = model.predict(X_test)

191/191 [==============================] - 0s 726us/step


In [145]:
roc_auc_score(y_test,predicts_for_test)

0.9983660769089857

In [146]:
# 0.9983660769089857